In [ ]:
!pip install neo4j


In [ ]:
!pip install psycopg2

In [6]:
#import the necessary libraries
from neo4j import GraphDatabase
import pandas as pd
import psycopg2
 
# Define Neo4j connection details

neo4j_uri = "neo4j+s://38cb2d98.databases.neo4j.io"
neo4j_user = "neo4j"
neo4j_password = "eLeMvqRIB-LEzL0m02uWsWzZAiBa1g7luK68YYD84W4"

# Define Postgres connection details
pg_host = "localhost"
pg_database = "neo4j_data"
pg_user = "postgres"
pg_password = "Admin1"

# Define Neo4j query to extract data
neo4j_query ="""
        MATCH (c:Customer)-[:HAS_SUBSCRIPTION]->(s:Subscription)-[:INCLUDES_SERVICE]->(se:Service)
        RETURN c.customer_id AS customer_id, s.subscription_id AS subscription_id, 
               se.service_id AS service_id, s.start_date AS start_date, 
               s.end_date AS end_date, s.price AS price
    """

# Define function to extract data from Neo4j and return a Pandas DataFrame
def extract_data(neo4j_uri, neo4j_user, neo4j_password, neo4j_query):
    # Connect to Neo4j
    driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user,neo4j_password))
    # Execute the query and retrieve the results
    with driver.session() as session:
        results = session.run(neo4j_query)
    # Convert results to pandas DataFrame
        df = pd.DataFrame(results.records, columns=results.keys())

    # Close the driver connection
    driver.close()

    return df  

# Define function to transform data
def transform_data(df):
    # Convert date fields to datetime objects
    df["start_date"] = pd.to_datetime(df["start_date"])
    df['end_date'] = pd.to_datetime(df['end_date'])
   
    # Remove null values
    df.dropna(inplace=True)
     
    return df

# Define function to load data into Postgres
def load_data(df):
    # Connect to Postgres
    conn = psycopg2.connect(host=pg_host, database=pg_database, user=pg_user, password=pg_password)
    # Create table if it doesn't exist
    with conn.cursor() as cursor:
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS telecom_data (
            customer_id INTEGER,
            subscription_id INTEGER,
            service_id INTEGER,
            start_date DATE,
            end_date DATE,
            price FLOAT
        )
        """)
    # Insert data into table
    cur = conn.cursor()
    for row in df.itertuples(index=False):
      cur.execute("""
            INSERT INTO subscriptions (customer_id, subscription_id, service_id, start_date, end_date, price)
            VALUES (%s, %s, %s, %s, %s, %s)
            """, row)

    conn.commit()
    conn.close()

def main():
    # Extract data from Neo4j
    data = extract_data()

    # Transform data using Pandas
    df = transform_data(df)

    # Load data into Postgres
    load_data(df)

if __name__ == "__main__":
    main()